# Save ADC traces from DOOCS to disk

### Imports

In [1]:
import pydoocs
import numpy as np
import time
import matplotlib.pyplot as plt
import h5py

### Parameters

In [2]:
adc_doocs_address = 'FLASH.FEL/ADC.ADQ.BL1/EXP1.CH00/CH00.DAQ.TD'
hdf_file_name = 'test'

train_ID = 0

### Utility functions

In [3]:
def wait_for_next_train():
    global train_ID
    while True:
        current_ID = pydoocs.read(adc_doocs_address)['macropulse']
        if train_ID != current_ID:
            train_ID = current_ID
            break

def time_axis_adc_values_waiting(adc_doocs_address):
    wait_for_next_train()
    time_value_matrix = pydoocs.read(adc_doocs_address)['data']
    time_axis = time_value_matrix[:, 0]
    adc_trace = time_value_matrix[:,1]
    return time_axis, adc_trace

def save_tof(adc_doocs_address, number_of_traces_to_record = 1):
    time_axis_tt = []
    adc_trace_tt = []
    train_ID_tt = []
    hf = h5py.File('../data/'+hdf_file_name+'.h5', 'w')
    for i in range(number_of_traces_to_record):
        time_axis, adc_trace = time_axis_adc_values_waiting(adc_doocs_address)
        time_axis_tt.append(time_axis)
        adc_trace_tt.append(adc_trace)
        train_ID_tt.append(train_ID)
    hf.create_dataset('train_ID_tt', data = train_ID_tt)
    hf.create_dataset('time_axis_tt', data = time_axis_tt)
    hf.create_dataset('adc_trace_tt', data = adc_trace_tt)
    hf.close()
    print((train_ID_tt[-1]-train_ID_tt[0]+1)/len(train_ID_tt))


In [4]:
save_tof(adc_doocs_address, 10)

NameError: name 'train_ID' is not defined

### check trainID in HDF file